# [가뭄지수 + 생산량 + 재배면적 + 종합유가 + 도매가] 데이터셋 작성

### import

In [ ]:
import os
import numpy as np
import pandas as pd
import statistics
import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime as dt 
import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

### 데이터셋 input

In [ ]:
## 예측가격 결정요인
가뭄지수 = pd.read_csv("../data/가뭄/2013_2020_가뭄지수데이터.csv", index_col = 0)
면세유가 = pd.read_csv("../data/유가/13_21_면세유가.csv", index_col = 0)
일반유가 = pd.read_csv("../data/유가/원본/13_21_주유소_지역별_평균판매가격.csv", encoding = "CP949")
면적생산 = pd.read_csv("../data/재배면적_생산량/2013_2020_주요지역_재배면적_생산량.csv", index_col = 0)
수입량 = pd.read_csv("../data/수입량/2013_2021월간배추수입량.csv", encoding = "CP949", names=["연도", "월", "수출(중량)", "수출(금액)", "수입(중량)", "수입(금액)"], thousands = ',')

## 관측지점 관련 데이터
addr = pd.read_excel('../data/지역/한국행정구역분류_2022.7.1.기준_20220630022706.xlsx', sheet_name = 1, header = [2, 2])
addr.drop('Unnamed: 0_level_0', axis = 1, inplace = True)
addr.columns = addr.columns.droplevel(1)
ori_coords = pd.read_excel('../data/지역/행정구역별_위경도_좌표.xlsx', sheet_name = None)
obPoints = pd.read_csv('../data/지역/관측지점_원본_기상청.csv')

### 태백시의 가뭄지수 결측치에 대한 대체값(최인접 지점-봉화군(결측)-울진군) 적용

In [ ]:
태백측정일자 = list()
for item in 가뭄지수[가뭄지수['시군명'] == '태백시']['발표일자'].unique() :
    태백측정일자.append(item)

가뭄지수_울진 = 가뭄지수[가뭄지수['시군명'] == '울진군']
가뭄지수_울진_추출 = 가뭄지수_울진[가뭄지수['발표일자'].apply(lambda x : x in 태백측정일자) == False]

## 추출된 울진지점 정보에 태백시 대입
가뭄지수_울진_추출['지점명'] = '태백'
가뭄지수_울진_추출['시군명'] = '태백시'
#가뭄지수_울진_추출['시도명'] = '강원'
가뭄지수_울진_추출

가뭄지수 = pd.concat([가뭄지수, 가뭄지수_울진_추출], ignore_index = True)
가뭄지수

## 1. 가뭄지수 + 종합유가(일반유가 + 면세유가)

In [ ]:
## 특이하게 저장된 부분들 편집
가뭄지수.replace('포항시남구',"포항시", inplace=True)
가뭄지수.replace('전주시덕진구',"전주시", inplace=True)
가뭄지수.replace('청주시흥덕구',"청주시", inplace=True)
가뭄지수.replace('\xa0서울특별시', '서울특별시', inplace = True)
가뭄지수.replace('\xa0인천광역시', '인천광역시', inplace = True)
가뭄지수.replace('\xa0대전광역시', '대전광역시', inplace = True)
가뭄지수.replace('\xa0대구광역시', '대구광역시', inplace = True)
가뭄지수.replace('\xa0울산광역시', '울산광역시', inplace = True)
가뭄지수.replace('\xa0광주광역시', '광주광역시', inplace = True)
가뭄지수.replace('\xa0부산광역시', '부산광역시', inplace = True)

### 1-1. 가뭄지수 관측지점의 시도명 추출

In [ ]:
## 시도명 컬럼 추가
## (time...)
가뭄지수['시도명'] = ''

## 시도명 탐색 후 대입
def helper(target) :
    try :
        ans = addr[addr['시군구'] == target].iloc[0]['시도']
    except :
        ans = addr[addr['시도'] == target].iloc[0]['시도']
        
    if ans[:2] in 면세유가.columns :
        return ans[:2]
    else :
        return (ans[0] + ans[2])

가뭄지수['시도명'] = 가뭄지수['시군명'].apply(lambda x : helper(x))
가뭄지수

### 1-2. 일반유가 merge

In [ ]:
## 일반유가 Merge
## (time...)

일반유가['구분'] = 일반유가['구분'].str.replace('년', '-')
일반유가['구분'] = 일반유가['구분'].str.replace('월', '-')
일반유가['구분'] = 일반유가['구분'].str.replace('일', '')
일반유가

## 컬럼으로 나누어진 지명을 로우로 변환
일반유가_월별정리 = pd.DataFrame()
for idx in 일반유가.index :
    for col in 일반유가.columns[1:] :
        temp = {'날짜' : 일반유가['구분'][idx], '지역' : col, '일반유가' : 일반유가[col][idx]}
        일반유가_월별정리 = pd.concat([일반유가_월별정리, pd.DataFrame([temp])])

## 가뭄지수-일반유가 Merge
가뭄지수_일반 = pd.merge(가뭄지수, 일반유가_월별정리, left_on = ['발표일자', '시도명'], right_on = ['날짜', '지역'], how = 'left')
가뭄지수_일반

### 1-3. 면세유가 merge

In [ ]:
## 면세유가 Merge
## (time...)

## 컬럼으로 나누어진 지명을 로우로 변환
면세유가_월별정리 = pd.DataFrame()
for idx in 면세유가.index :
    for col in 면세유가.columns[1:] :
        temp = {'날짜' : 면세유가['날짜'][idx], 
                '지역' : col, 
                '면세유가' : 면세유가[col][idx]}
        면세유가_월별정리 = pd.concat([면세유가_월별정리, pd.DataFrame([temp])])
면세유가_월별정리

## 가뭄지수-면세유가 Merge
가뭄지수_종합유가 = pd.merge(가뭄지수_일반, 면세유가_월별정리, left_on = ['날짜', '지역'], right_on = ['날짜', '지역'], how = 'left')
가뭄지수_종합유가

## 2. (가뭄지수+종합유가) + 면적생산

In [ ]:
## 연도 컬럼 추가
가뭄지수_종합유가['연도'] = ''
가뭄지수_종합유가['연도'] = 가뭄지수_종합유가['발표일자'].apply(lambda x : x[:4])
가뭄지수_종합유가

### 2-1. 각 배추 품종별 최대면적 row 추출

In [ ]:
# 계절별 구분 필요
최대값 = pd.DataFrame()
for season in 면적생산['품명'].unique() :
    temp = pd.DataFrame(면적생산[면적생산['품명'] == season].groupby(['연도', '품명'])['면적'].max()).reset_index()
    최대값 = pd.concat([최대값, temp] , ignore_index = True)
최대값

### 2-2. 최대면적 해당 지역 및 생산량 탐색

In [ ]:
면적생산_최대면적 = pd.DataFrame()
for idx in 최대값.index :
    
    merged = 면적생산[(면적생산['연도'] == 최대값['연도'][idx]) & 
                   (면적생산['품명'] == 최대값['품명'][idx]) &
                   (면적생산['면적'] == 최대값['면적'][idx])]
    
    면적생산_최대면적 = pd.concat([면적생산_최대면적, merged] , ignore_index = True)
면적생산_최대면적['지역'] = 면적생산_최대면적['지역'].replace('예산시', '예산군')

### 2-3. 관측지점 대체 지역 리스트 작성

In [ ]:
## 최대면적 해당 지역과 관측지점 불일치로 인한 대체 지역 리스트
noItemLi = list()
for item in 면적생산_최대면적['지역'].unique()  :
    if not item in 가뭄지수_종합유가['시군명'].unique() :
        noItemLi.append(item)

## 관측지점 불일치 지점 목록 DF화
noItemDF = pd.DataFrame(noItemLi, columns = ['시군구'])

## 개정 지명 예외처리
noItemDF.replace('마산시', '창원시', inplace = True)
noItemDF.replace('진해시', '창원시', inplace = True)
noItemDF.replace('예산시', '예산군', inplace = True)
noItemDF.replace('세종특별시', '세종특별자치시', inplace = True)

noItemDF

### 2-4. 관측지점 대체를 위한 지점별 좌표 정보 탐색

In [ ]:
pd.concat(ori_coords, ignore_index = True)

In [ ]:
## 행정구역별 좌표 자료
ori_coords = pd.concat(ori_coords, ignore_index = True)

## 시군구 단위 추출
coords = ori_coords

dropidx = coords[coords['읍면동/구'].isna() == False].index
coords.drop(dropidx, inplace = True)
coords.drop(['읍면동/구', '읍/면/리/동', '리'], axis = 1, inplace = True)
coords.reset_index(drop = True, inplace =True)

#coords.to_csv("./data/지역/전국행정구역좌표정리.csv", encoding = 'utf-8-sig')

coords['시군구'].fillna(coords['시도'], inplace = True)
coords['시군구'].str.strip()

coords

### 2-5. 관측지점 불일치 지점의 좌표 정보 mapping

In [ ]:
noItemCoordsDF = pd.merge(noItemDF, coords, on = '시군구', how = 'left').drop('시도', axis = 1)
noItemCoordsDF['대체지점명'] = ''
noItemCoordsDF

### 2-6. 기상청 관측지점 좌표 정보 탐색

In [ ]:
## 기상청 관측지점 정보
obPoints.dropna(inplace = True)
obPoints.drop(columns = ['종료시각', '시작시각', '노장 해발고도(m)','기압계', '기온계', '풍속계', '강우계'], inplace = True)
obPoints.columns = ['지점코드', '지점명', '주소', '위도', '경도']
obPoints.reset_index(drop = True)

obPoints.주소 = obPoints.주소.str.split(" ")
region = []
for item in obPoints["주소"]:
    first = item[0]
    second = item[1]
    
    if "시" in second:
        region.append(second)
    elif "군" in second:
        region.append(second)
    elif "시" in first:
        region.append(first)
        
obPoints["주소"] = region

## 원본 데이터 내 이상 데이터 편집
obPoints.replace('포항시남구',"포항시", inplace=True)
obPoints.replace('전주시덕진구',"전주시", inplace=True)
obPoints.replace('청주시흥덕구',"청주시", inplace=True)
obPoints.replace('\xa0서울특별시', '서울특별시', inplace = True)
obPoints.replace('\xa0인천광역시', '인천광역시', inplace = True)
obPoints.replace('\xa0대전광역시', '대전광역시', inplace = True)
obPoints.replace('\xa0대구광역시', '대구광역시', inplace = True)
obPoints.replace('\xa0울산광역시', '울산광역시', inplace = True)
obPoints.replace('\xa0광주광역시', '광주광역시', inplace = True)
obPoints.replace('\xa0부산광역시', '부산광역시', inplace = True)

## 전체 기상청 관측지점 중 가뭄지수 관측 지점에 해당하는 68개 지점 추출
def helper(x) :
    if x in list(가뭄지수['시군명'].unique()) :
        return x
    else :
        np.nan

obPoints['주소'] = obPoints['주소'].apply(lambda x : helper(x))
obPoints.dropna(inplace = True)
obPoints.reset_index(drop = True, inplace = True)

obPoints

### 2-7. 최단거리내의 대체 관측지점 탐색 및 대입

In [ ]:
def dist(x_long, x_lati) :
    minLi = list()
    
    for idx in obPoints.index :
        long = pow(float(x_long) - float(obPoints.loc[idx]['경도']), 2)
        lati = pow(float(x_lati) - float(obPoints.loc[idx]['위도']), 2)
        minLi.append([np.sqrt(long + lati), obPoints.loc[idx]['주소']])
        
    return minLi[minLi.index(min(minLi))][1]

noItemCoordsDF['대체지점명'] = noItemCoordsDF.apply(lambda x : dist(x['경도'], x['위도']), axis = 1)
noItemCoordsDF.drop_duplicates(keep = 'first', inplace = True)
noItemCoordsDF

### 2-8. 대체관측지점 추가

In [ ]:
면적생산_대체지점 = pd.merge(면적생산_최대면적, noItemCoordsDF, left_on = '지역', right_on = '시군구', how = 'left')
면적생산_대체지점 = 면적생산_대체지점[['연도', '품명', '지역', '면적', '생산량', '대체지점명']]

## 대체지점있을 경우 적용, 없으면 기존 관측지점을 그대로 적용
def helper(x_ori, x_rep) :
    if pd.isna(x_rep) :
        return x_ori
    else :
        return x_rep

면적생산_대체지점['관측지점명'] = 면적생산_대체지점.apply(lambda x : helper(x['지역'], x['대체지점명']), axis = 1)
면적생산_대체지점.drop(['대체지점명'], axis = 1, inplace = True)
면적생산_대체지점.columns = ['연도', '품명', '지명' ,'면적', '생산량', '관측지점명']
면적생산_대체지점['연도'] = 면적생산_대체지점['연도'].apply(lambda x : str(x))
면적생산_대체지점

### 2-9. 면적생산 merge

In [ ]:
## 가뭄지수_종합유가 DF - 면적생산 DF Merge
가뭄지수_종합유가_면적생산 = pd.merge(가뭄지수_종합유가, 면적생산_대체지점, left_on = ['연도', '시군명'], right_on = ['연도', '관측지점명'], how = 'right')

가뭄지수_종합유가_면적생산.drop(['연도'], axis = 1, inplace = True)
가뭄지수_종합유가_면적생산 = 가뭄지수_종합유가_면적생산[['발표일자', '품명', '지점명', '시군명', '시도명', '지역', '관측지점명', '일반유가', '면세유가', '지명', '면적', '생산량', 'SPI1', 'SPI3', 'EDI']]
가뭄지수_종합유가_면적생산

# 3. (가뭄지수+종합유가 + 면적생산) + 수입량액

### 3-1. 수입량액 자료열 정리

In [ ]:
수입량.drop(['수출(중량)', '수출(금액)', '수입(금액)'], axis = 1, inplace = True)
수입량.drop(수입량[수입량['월'] == '합'].index, inplace = True)
수입량.columns = ['연도', '월', '수입중량']

수입량['연도'] = 수입량['연도'].apply(lambda x : str(x))
수입량['월'] = 수입량['월'].apply(lambda x : str(x))

수입량['연월'] = 수입량['연도'] + '-' + 수입량['월']
수입량.drop(['연도', '월'], axis = 1, inplace = True)
수입량 = 수입량[['연월', '수입중량']]
수입량

### 3-2. 수입량액 연월 정보 추가 및 발표일자 str화

In [ ]:
가뭄지수_종합유가_면적생산['발표일자'] = 가뭄지수_종합유가_면적생산['발표일자'].apply(lambda x : str(x))
가뭄지수_종합유가_면적생산['연월'] = 가뭄지수_종합유가_면적생산['발표일자'].apply(lambda x : x[:7])

### 3-3. 수입량액 merge

In [ ]:
가뭄지수_종합유가_면적생산_수입량액 = pd.merge(가뭄지수_종합유가_면적생산, 수입량, on = '연월', how = 'left')

가뭄지수_종합유가_면적생산_수입량액.drop(['연월'], axis = 1, inplace = True)
가뭄지수_종합유가_면적생산_수입량액 = 가뭄지수_종합유가_면적생산_수입량액[['발표일자', '품명', '지점명', '시군명', '시도명', '지역', '지명', '관측지점명', '일반유가', '면세유가',  '면적','생산량', '수입중량', 'SPI1', 'SPI3', 'EDI']]

가뭄지수_종합유가_면적생산_수입량액

# 4. (가뭄지수+면세유가 + 면적생산 + 수입량액) + 도매가(지역별)

### 4-1. 지역별 최종학습데이터셋 파일 저장

In [ ]:
최종 = list()
for loc in ['서울', '부산', '대구', '광주', '대전'] :
    df = pd.read_csv(f'../data/도매가/보간/13_21_{loc}_도매가보간.csv', encoding = 'utf-8-sig', index_col = 0)
    df.drop('지역', axis = 1, inplace = True)
    
    최종학습데이터 = pd.merge(가뭄지수_종합유가_면적생산_수입량액, df, left_on = ['발표일자', '품명'], right_on = ['날짜', '품명'], how = 'right')
    최종학습데이터.dropna(inplace = True)
    최종학습데이터.drop(['날짜'], axis = 1, inplace = True)
    
    최종학습데이터.rename(columns = {'발표일자' : '날짜'}, inplace = True)
    최종학습데이터 = 최종학습데이터[['날짜', '품명', '지역', '지명', '관측지점명', '일반유가', '면세유가', '면적', '생산량', '수입중량', 'SPI1', 'SPI3', 'EDI', '도매가격']].reset_index(drop = True)
    
    최종학습데이터.to_csv(f'../data/result/입력/학습데이터1차_{loc}.csv', encoding = 'utf-8-sig')
    최종.append(최종학습데이터)

In [ ]:
최종[0]

In [ ]:
최종[0].info()

### 최종학습데이터셋 조정 시 필요 데이터셋 저장

In [ ]:
가뭄지수.to_csv("../data/result/입력/가뭄지수_최종정리.csv", encoding = "utf-8-sig")
일반유가_월별정리.to_csv("../data/result/입력/일반유가_최종정리.csv", encoding = "utf-8-sig")
면세유가_월별정리.to_csv("../data/result/입력/면세유가_최종정리.csv", encoding = "utf-8-sig")
면적생산_대체지점.to_csv("../data/result/입력/면적생산_최종정리.csv", encoding = "utf-8-sig")
수입량.to_csv("../data/result/입력/수입량_최종정리.csv", encoding = "utf-8-sig")